In [1]:
import pandas as pd
from string import printable
from sklearn.cross_validation import train_test_split
df1=pd.read_csv("urlset.csv",encoding='latin-1')

C:\Users\40750\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
domain=list(df1['domain'])
labels=list(df1['label'])
for i in domain:
    if i.startswith('\'') and i.endswith('\''):
        indx=domain.index(i)
        i=i.replace('\'','')
        domain[indx]=i 

In [5]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
#from keras.initializers import glorot_uniform
from keras.preprocessing.text import Tokenizer
from keras import regularizers

C:\Users\40750\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [31]:
max_len = 75
url_int_tokens = [[printable.index(x) + 1 for x in d if x in printable] for d in domain]
X = sequence.pad_sequences(url_int_tokens, maxlen=max_len)
print(type(X))
X_train, X_test, y_train, y_test=train_test_split(X,labels,test_size=0.25,random_state=42)

<class 'numpy.ndarray'>


In [12]:
def phising():
    main_input = Input(shape=(max_len,), dtype='int32', name='main_input')
    emb = Embedding(input_dim=100, output_dim=32, input_length=max_len,
                        dropout=0.2, W_regularizer=regularizers.l2(1e-4))(main_input)
    X = LSTM(32)(emb)
    X =  Dropout(0.5)(X)
    X = Dense(1, activation='sigmoid', name='output')(X)
    model = Model(inputs=[main_input], outputs=X)
    return model

In [13]:
model=phising()

C:\Users\40750\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
C:\Users\40750\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(input_dim=100, output_dim=32, input_length=75, embeddings_regularizer=<keras.reg...)`
  


In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(X_train, y_train, epochs=30, batch_size=1000, verbose=1)

Epoch 1/30
71931/71931 [==============================] - 30s 414us/step - loss: 0.5703 - acc: 0.7100
Epoch 2/30
71931/71931 [==============================] - 27s 381us/step - loss: 0.3931 - acc: 0.8264
Epoch 3/30
71931/71931 [==============================] - 27s 379us/step - loss: 0.3475 - acc: 0.8512
Epoch 4/30
71931/71931 [==============================] - 26s 362us/step - loss: 0.3274 - acc: 0.8612
Epoch 5/30
71931/71931 [==============================] - 25s 354us/step - loss: 0.3102 - acc: 0.8688
Epoch 6/30
71931/71931 [==============================] - 25s 351us/step - loss: 0.2980 - acc: 0.8766
Epoch 7/30
71931/71931 [==============================] - 26s 357us/step - loss: 0.2844 - acc: 0.8831
Epoch 8/30
71931/71931 [==============================] - 25s 351us/step - loss: 0.2733 - acc: 0.8900
Epoch 9/30
71931/71931 [==============================] - 26s 361us/step - loss: 0.2634 - acc: 0.8941
Epoch 10/30
71931/71931 [==============================] - 26s 355us/step - loss: 

In [16]:
model.evaluate(X_test, y_test, verbose=1)

23977/23977 [==============================] - 9s 363us/step


[0.1605102640738672, 0.9392751386745631]

In [30]:
def predict(x_input):
    #print(x_input)
    url_int_tokens_1=[[printable.index(x) + 1 for x in x_input if x in printable]]
    #print(url_int_tokens_1)
    X_1 = sequence.pad_sequences(url_int_tokens_1, maxlen=max_len)
    p = model.predict(X_1)
    print(p)
    #print(model.predict(X_1))
    return "not_malicious" if p < 0.6 else "malicious"
x_input="https://lnkd.in/dkk5-Ti"
print(predict(x_input))

[[0.9775122]]
malicious
